In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/womens-e-commerce-clothing-reviews/Women_s_E-Commerce_Clothing_Reviews_1594_1.csv


**Read CSV files with semicolon separator**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. Membaca file CSV dengan pemisah titik koma
df = pd.read_csv('/kaggle/input/womens-e-commerce-clothing-reviews/Women_s_E-Commerce_Clothing_Reviews_1594_1.csv', sep=';', on_bad_lines='skip')

**Data preview.**

**Clean data (delete rows with missing values ​​in 'Review.Text' and 'Rating' columns).**

In [3]:
# 2. Pratinjau data
print(df.head())

# 3. Bersihkan data (menghapus baris dengan nilai yang hilang di kolom 'Review.Text' dan 'Rating')
df.dropna(subset=['Review.Text', 'Rating'], inplace=True)

   Unnamed: 0  Clothing.ID  Age                    Title  \
0           1          767   33                      NaN   
1           2         1080   34                      NaN   
2           3         1077   60  Some major design flaws   
3           4         1049   50         My favorite buy!   
4           5          847   47         Flattering shirt   

                                         Review.Text  Rating  Recommended.IND  \
0  Absolutely wonderful - silky and sexy and comf...       4                1   
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
3  I love, love, love this jumpsuit. it's fun, fl...       5                1   
4  This shirt is very flattering to all due to th...       5                1   

   Positive.Feedback.Count   Division.Name Department.Name Class.Name  
0                        0       Initmates        Intimate  Intimates  
1       

**Text processing**

In [4]:
# 4. Pemrosesan teks
X = df['Review.Text']
y = df['Rating']

**Categorize ratings into 3 classes: negative (1-2), neutral (3), positive (4-5)**

In [5]:
# Mengkategorikan rating menjadi 3 kelas: negatif (1-2), netral (3), positif (4-5)
y = y.apply(lambda x: 'positive' if x > 3 else ('negative' if x < 3 else 'neutral'))

**Feature extraction using TF-IDF**

In [6]:
# 5. Ekstraksi fitur menggunakan TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

**Split data into training and test sets**

In [7]:
# 6. Membagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

**Training model classification**

In [8]:
# 7. Melatih model klasifikasi
model = LogisticRegression()
model.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

**Model evaluation**

In [9]:
# 8. Evaluasi model
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Accuracy: 0.8165157871494811
              precision    recall  f1-score   support

    negative       0.56      0.41      0.48       457
     neutral       0.46      0.20      0.28       588
    positive       0.86      0.97      0.91      3484

    accuracy                           0.82      4529
   macro avg       0.63      0.53      0.56      4529
weighted avg       0.78      0.82      0.79      4529



**Added a sentiment prediction column to the original DataFrame**

In [10]:
# 9. Menambahkan kolom prediksi sentimen ke DataFrame asli
df['Sentiment'] = df['Review.Text'].apply(lambda text: model.predict(vectorizer.transform([text]))[0])

**Save the DataFrame with the prediction column to a new file**

In [11]:
# 10. Menyimpan DataFrame dengan kolom prediksi ke file baru
df.to_csv('Women_s_E-Commerce_Clothing_Reviews_with_Sentiment.csv', sep=';', index=False)

print("Hasil prediksi sentimen telah disimpan ke 'Women_s_E-Commerce_Clothing_Reviews_with_Sentiment.csv'.")

Hasil prediksi sentimen telah disimpan ke 'Women_s_E-Commerce_Clothing_Reviews_with_Sentiment.csv'.
